# Exploring inference methods to plan inference pipeline

In [1]:
import hopsworks
from datetime import datetime
import pandas as pd

/Users/sambook/miniconda3/envs/id2223/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Today's date ----- is this needed/helpful anywhere?
today = datetime.now().strftime('%Y-%m-%d')
print(type(today))
print(today)

<class 'str'>
2023-12-14


## Get stored news articles

In [5]:
project = hopsworks.login()
fs = project.get_feature_store()

news_fg = fs.get_feature_group(name="news_articles", version=1)
news_df = news_fg.read()
news_df.head()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/185495
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (1.08s) 


,article_id,title,link,description,content,pubdate,source_id,source_priority,country,category,language
0,614fe26d75302581ff4c727dc2b62069,AP Breakthrough Entertainer: Charles Melton th...,https://www.winnipegfreepress.com/arts-and-lif...,LOS ANGELES (AP) — Fame didn’t come overnight ...,LOS ANGELES (AP) — Fame didn’t come overnight ...,2023-12-13 14:13:27,winnipegfreepress,101538,[canada],[entertainment],english
1,c1a910e57b0588354924cb70c07ff63e,"Badly burned but free, Israa Jaabis on her rel...",https://www.aljazeera.com/features/2023/12/13/...,Israa Jaabis was accused of attempted murder w...,Israa Jaabis was accused of attempted murder w...,2023-12-13 14:13:33,aljazeera_us,543,[united states of america],[top],english
2,975eafb21824c494b7b5492c189c397b,Dick Van Dyke Through the Years: Photos of the...,https://hollywoodlife.com/pics/dick-van-dyke-p...,Dick Van Dyke is one of Hollywood's most celeb...,Dick Van Dyke is one of Hollywood’s most celeb...,2023-12-13 14:15:32,hollywoodlife,977,[united states of america],[world],english
3,1cf88a47fd65f8a9b8be421268af0f4a,Virginia has tentative deal to move Washington...,https://lethbridgenewsnow.com/2023/12/13/virgi...,"RICHMOND, Va. (AP) - Virginia Gov. Glenn Young...","RICHMOND, Va. (AP) — Virginia Gov. Glenn Young...",2023-12-13 14:07:18,lethbridgenewsnow,149548,[canada],[top],english
4,2170ae2a33b0d42be841601f63d95311,Sydney Sweeney admits to being a 'workaholic',https://www.perthnow.com.au/entertainment/cele...,Sydney Sweeney has revealed that she's current...,"Sydney Sweeney is a ""workaholic"". The 26-year-...",2023-12-13 14:00:10,perthnow,49457,[australia],[entertainment],english


In [9]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [30]:
def format_sentiment(sentiment):
    if sentiment['label'] == 'NEGATIVE':
        return sentiment['score'] * -1
    else:
        return sentiment['score']

def get_sentiment_value(news_object):
    return format_sentiment(sentiment_pipeline(news_object['title'])[0])

In [31]:
news_df['sentiment'] = news_df.apply(get_sentiment_value, axis=1)

In [44]:
news_df.head()

,article_id,title,link,description,content,pubdate,source_id,source_priority,country,category,language,sentiment
0,614fe26d75302581ff4c727dc2b62069,AP Breakthrough Entertainer: Charles Melton th...,https://www.winnipegfreepress.com/arts-and-lif...,LOS ANGELES (AP) — Fame didn’t come overnight ...,LOS ANGELES (AP) — Fame didn’t come overnight ...,2023-12-13 14:13:27,winnipegfreepress,101538,[canada],[entertainment],english,0.999470
1,c1a910e57b0588354924cb70c07ff63e,"Badly burned but free, Israa Jaabis on her rel...",https://www.aljazeera.com/features/2023/12/13/...,Israa Jaabis was accused of attempted murder w...,Israa Jaabis was accused of attempted murder w...,2023-12-13 14:13:33,aljazeera_us,543,[united states of america],[top],english,-0.861551
2,975eafb21824c494b7b5492c189c397b,Dick Van Dyke Through the Years: Photos of the...,https://hollywoodlife.com/pics/dick-van-dyke-p...,Dick Van Dyke is one of Hollywood's most celeb...,Dick Van Dyke is one of Hollywood’s most celeb...,2023-12-13 14:15:32,hollywoodlife,977,[united states of america],[world],english,0.990084
3,1cf88a47fd65f8a9b8be421268af0f4a,Virginia has tentative deal to move Washington...,https://lethbridgenewsnow.com/2023/12/13/virgi...,"RICHMOND, Va. (AP) - Virginia Gov. Glenn Young...","RICHMOND, Va. (AP) — Virginia Gov. Glenn Young...",2023-12-13 14:07:18,lethbridgenewsnow,149548,[canada],[top],english,0.972165
4,2170ae2a33b0d42be841601f63d95311,Sydney Sweeney admits to being a 'workaholic',https://www.perthnow.com.au/entertainment/cele...,Sydney Sweeney has revealed that she's current...,"Sydney Sweeney is a ""workaholic"". The 26-year-...",2023-12-13 14:00:10,perthnow,49457,[australia],[entertainment],english,-0.981958
